In [1]:
import requests
import json

In [3]:
data = {
    'prompt': '[INST] Who is Elon Musk? [/INST]',
    'sampling_params' : {
        'temperature': 0.9,
    }
}

url = 'http://127.0.0.1:8000/llm/generate'

# response = requests.post(url, data={'body': json.dumps(data)})
response = requests.post(url, json=data)
response.json()

{'vllm_llama2_7b_chat_output': '  Elon Musk is a South African-born entrepreneur, inventor, and business magnate. He is best known for his involvement in revolutionizing multiple industries through his companies, including transportation, energy, and space exploration. Here are some key facts about Elon Musk:\n\n1. Early Life and Education: Musk was born on June 28, 1971, in Pretoria, South Africa. He developed an interest in computing and programming at an early age and taught himself computer programming. He moved to Canada in 1992 to attend college, and later transferred to the University of Pennsylvania, where he graduated with a degree in economics and physics.\n2. Entrepreneurial Career: Musk co-founded his first company, Zip2, which provided online content publishing software for news organizations. In 1999, he co-founded X.com, which later became PayPal, an online payment system that was acquired by eBay for $1.5 billion in 2002.\n3. SpaceX: In 2002, Musk founded SpaceX, a priv